In [1]:
def scale_features(X, scalers):
    X_scaled = X.copy()
    for feature in X.columns:
        X_scaled[feature] = scalers[feature].transform(X[[feature]])
    return X_scaled

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, classification_report
from interpret.glassbox import ExplainableBoostingClassifier
from itertools import product
import joblib

df = pd.read_csv("heloc_preprocessed.csv")

simple_feature_names = ["Overall Credit Risk Score", "Months Since First Credit Account", "Average Age of Credit Accounts", "Number of Well-Maintained Accounts", "Percentage of Accounts Never Late",
                            "Months Since Last Missed Payment", "Percentage of Installment vs Revolving Loans", "Time Since Last Credit Application", "Credit Utilization Ratio", "Number of Active Credit Cards/Lines", "Loan Repaid"]

df_simple = df.copy()
df_simple.columns = simple_feature_names

y = df_simple["Loan Repaid"]
X = df_simple.drop(columns="Loan Repaid")

scalers = {feature: StandardScaler() for feature in X.columns}

n_folds = 5
random_state = 42

model_name = "EBM"
# Hyperparameter search space
ebm_hyperparameters = {
    "max_bins": [256, 512],
    "interactions": [0, 10, 20],
    "outer_bags": [8, 16],
    "inner_bags": [0, 4],
    "random_state": [random_state]
}

overall_best_hp_config = None
overall_best_loss = np.inf
best_model = None  # Store the best model overall

# Split off final test set for dashboard accuracy calculation
X_train_full, X_final_test, y_train_full, y_final_test = train_test_split(
    X, y, test_size=0.15, stratify=y, random_state=random_state
)

for feature in X.columns:
    scalers[feature].fit(X_train_full[[feature]])  # Only fit on the training set

X_train_full_scaled = scale_features(X_train_full, scalers)
X_final_test_scaled = scale_features(X_final_test, scalers)

# Use KFold (not stratified) since dataset is fairly balanced
outer_cv = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)

for fold_i, (train_val_idx, test_idx) in enumerate(outer_cv.split(X_train_full_scaled, y_train_full)):
    print(f"\n----- Model: {model_name} -- Fold: {fold_i + 1}/{n_folds} -----")

    X_train_val_scaled, y_train_val = X_train_full_scaled.iloc[train_val_idx], y_train_full.iloc[train_val_idx]
    X_test_scaled, y_test = X_train_full_scaled.iloc[test_idx], y_train_full.iloc[test_idx]

    # Split the training data into train and validation sets
    X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(
        X_train_val_scaled, y_train_val, test_size=0.25, stratify=y_train_val, random_state=random_state
    )

    # Grid search over hyperparameters
    best_hp_config = None
    best_loss = np.inf

    for hp in product(*ebm_hyperparameters.values()):
        params = dict(zip(ebm_hyperparameters.keys(), hp))
        model = ExplainableBoostingClassifier(**params)
        model.fit(X_train_scaled, y_train)

        y_val_pred_proba = model.predict_proba(X_val_scaled)
        ce_loss = log_loss(y_val, y_val_pred_proba)

        if ce_loss < best_loss:
            best_loss = ce_loss
            best_hp_config = params
            best_model = model  # Store the best model found

    # After the grid search, we select the best model and print hyperparameters
    print(f"Best hyperparameters: {best_hp_config}")

    # Inside the cross-validation loop, update this:
    if best_loss < overall_best_loss:
        overall_best_loss = best_loss
        overall_best_hp_config = best_hp_config  # Store best config across folds

# Retain the best-trained model instead of retraining from scratch
final_model = best_model  # Use the best model found during CV

# Predict on final test set
y_final_pred = final_model.predict(X_final_test_scaled)

# Compute accuracy & metrics
print(classification_report(y_final_test, y_final_pred))

# Save model
joblib.dump(final_model, "final_ebm_standardized_model.pkl")
joblib.dump(scalers, "scalers.pkl")



----- Model: EBM -- Fold: 1/5 -----
Best hyperparameters: {'max_bins': 256, 'interactions': 10, 'outer_bags': 8, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 2/5 -----
Best hyperparameters: {'max_bins': 512, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 3/5 -----
Best hyperparameters: {'max_bins': 512, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 4/5 -----
Best hyperparameters: {'max_bins': 512, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 5/5 -----
Best hyperparameters: {'max_bins': 256, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 0, 'random_state': 42}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       819
           1       0.70      0.73      0.71       750

    accuracy                           0.72      1569
   macro avg       0.72

['scalers.pkl']

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss, classification_report
from interpret.glassbox import ExplainableBoostingClassifier
from itertools import product
import joblib

df = pd.read_csv("heloc_preprocessed.csv")

simple_feature_names = ["Overall Credit Risk Score", "Months Since First Credit Account", "Average Age of Credit Accounts", "Number of Well-Maintained Accounts", "Percentage of Accounts Never Late",
                            "Months Since Last Missed Payment", "Percentage of Installment vs Revolving Loans", "Time Since Last Credit Application", "Credit Utilization Ratio", "Number of Active Credit Cards/Lines", "Loan Repaid"]

df_simple = df.copy()
df_simple.columns = simple_feature_names

y = df_simple["Loan Repaid"]
X = df_simple.drop(columns="Loan Repaid")

n_folds = 5
random_state = 42

model_name = "EBM"
# Hyperparameter search space
ebm_hyperparameters = {
    "max_bins": [256, 512],
    "interactions": [0, 10, 20],
    "outer_bags": [8, 16],
    "inner_bags": [0, 4],
    "random_state": [random_state]
}

overall_best_hp_config = None
overall_best_loss = np.inf
best_model = None  # Store the best model overall

# Split off final test set for dashboard accuracy calculation
X_train_full, X_final_test, y_train_full, y_final_test = train_test_split(
    X, y, test_size=0.15, stratify=y, random_state=random_state
)

# Use KFold (not stratified) since dataset is fairly balanced
outer_cv = KFold(n_splits=n_folds, shuffle=True, random_state=random_state)

for fold_i, (train_val_idx, test_idx) in enumerate(outer_cv.split(X_train_full, y_train_full)):
    print(f"\n----- Model: {model_name} -- Fold: {fold_i + 1}/{n_folds} -----")

    # Split train-validation-test
    X_train_val, y_train_val = X_train_full.iloc[train_val_idx], y_train_full.iloc[train_val_idx]
    X_test, y_test = X_train_full.iloc[test_idx], y_train_full.iloc[test_idx]

    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=random_state
    )

    # Grid search over hyperparameters
    best_hp_config = None
    best_loss = np.inf

    for hp in product(*ebm_hyperparameters.values()):
        params = dict(zip(ebm_hyperparameters.keys(), hp))
        model = ExplainableBoostingClassifier(**params)
        model.fit(X_train, y_train)

        y_val_pred_proba = model.predict_proba(X_val)
        ce_loss = log_loss(y_val, y_val_pred_proba)

        if ce_loss < best_loss:
            best_loss = ce_loss
            best_hp_config = params
            best_model = model  # Store the best model found

    # Train final model on full train-val data with best params
    print(f"Best hyperparameters: {best_hp_config}")

    # Inside the cross-validation loop, update this:
    if best_loss < overall_best_loss:
        overall_best_loss = best_loss
        overall_best_hp_config = best_hp_config  # Store best config across folds

# Retain the best-trained model instead of retraining from scratch
final_model2 = best_model  # Use the best model found during CV

# Predict on final test set
y_final_pred = final_model2.predict(X_final_test)

# Compute accuracy & metrics
print(classification_report(y_final_test, y_final_pred))

# Save model
joblib.dump(final_model2, "final_ebm_model.pkl")



----- Model: EBM -- Fold: 1/5 -----
Best hyperparameters: {'max_bins': 256, 'interactions': 10, 'outer_bags': 8, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 2/5 -----
Best hyperparameters: {'max_bins': 512, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 3/5 -----
Best hyperparameters: {'max_bins': 512, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 4/5 -----
Best hyperparameters: {'max_bins': 512, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 4, 'random_state': 42}

----- Model: EBM -- Fold: 5/5 -----
Best hyperparameters: {'max_bins': 256, 'interactions': 20, 'outer_bags': 16, 'inner_bags': 0, 'random_state': 42}
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       819
           1       0.70      0.73      0.71       750

    accuracy                           0.72      1569
   macro avg       0.72

['final_ebm_model.pkl']

In [4]:
from interpret import show
ebm_standard = joblib.load("final_ebm_standardized_model.pkl")
show(ebm_standard.explain_global())

<!-- http://127.0.0.1:7001/140446342494432/ -->

In [5]:
ebm = joblib.load("final_ebm_model.pkl")
show(ebm.explain_global())

<!-- http://127.0.0.1:7001/140446377469168/ -->